In [2]:
from operator import itemgetter

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

model = ChatOpenAI(api_key="{YOUR_OPENAI_KEY}")

chain1 = prompt1 | model | StrOutputParser()

chain2 = (
    {"city": chain1, "language": itemgetter("language")}
    | prompt2
    | model
    | StrOutputParser()
)

output = chain2.invoke({"person": "obama", "language": "english"})
print(output)

Chicago, Illinois, is located in the United States.


In [6]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

prompt1 = ChatPromptTemplate.from_template(
    "what is the famous tour place in {city}? Tell me the name of the place only without additional comments."
)
prompt2 = ChatPromptTemplate.from_template("How can I get {place} by {transport}?")

model = ChatOpenAI(api_key="{YOUR_OPENAI_KEY}")

chain1 = prompt1 | model | StrOutputParser()

print("chain 1:", chain1.invoke({"city": "Seoul"}))

chain2 = (
    {"place": chain1, "transport": itemgetter("transport")}
    | prompt2
    | model
    | StrOutputParser()
)

output = chain2.invoke({"city": "Seoul", "transport": "subway"})
print(output)

chain 1: Gyeongbokgung Palace
To get to Gyeongbokgung Palace by subway in Seoul, you can follow these steps:

1. **Find the Nearest Subway Station**: The closest subway station to Gyeongbokgung Palace is **Gyeongbokgung Station** on Line 3 (the Orange Line).

2. **Take the Subway**:
   - If you are starting from **Incheon International Airport**, take the Airport Railroad Express (AREX) to **Seoul Station**, then transfer to Line 4 (the Blue Line) and go to **Gwanghwamun Station**. From there, you can walk to the palace.
   - If you are starting from **Seoul Station**, take Line 1 (the Dark Blue Line) to **City Hall Station**, then transfer to Line 2 (the Green Line) and go to **Gyeongbokgung Station**.
   - If you are already on Line 3, simply take the train to **Gyeongbokgung Station**.

3. **Exit the Station**: Once you arrive at Gyeongbokgung Station, use Exit 5. The palace is a short walk from the station.

4. **Walking Directions**: After exiting, you will see signs directing you

# Pipe를 이용한 Chaining Sample


In [8]:
from langchain_core.runnables import RunnablePassthrough

chat = ChatOpenAI(api_key="{YOUR_OPENAI_KEY}")
planner = (
    ChatPromptTemplate.from_template("Generate an argument about: {input}")
    | chat
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

arguments_for = (
    ChatPromptTemplate.from_template(
        "List the pros or positive aspects of {base_response}"
    )
    | chat
    | StrOutputParser()
)
arguments_against = (
    ChatPromptTemplate.from_template(
        "List the cons or negative aspects of {base_response}"
    )
    | chat
    | StrOutputParser()
)

final_responder = (
    ChatPromptTemplate.from_messages(
        [
            ("ai", "{original_response}"),
            ("human", "Pros:\n{results_1}\n\nCons:\n{results_2}"),
            ("system", "Generate a final response given the critique"),
        ]
    )
    | chat
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": arguments_for,
        "results_2": arguments_against,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)

output = chain.invoke({"input": "scrum"})
print(output)

**Final Response: Evaluating the Pros and Cons of Scrum as a Project Management Framework**

Scrum has emerged as a popular project management framework, particularly in agile environments, due to its numerous advantages. However, it is essential to recognize that, like any methodology, it comes with its own set of challenges. A balanced evaluation of both the pros and cons can help organizations make informed decisions about adopting Scrum.

### Pros of Scrum

1. **Enhanced Collaboration and Communication:** 
   Scrum fosters a culture of transparency and teamwork through regular ceremonies, which can lead to innovative solutions and improved morale.

2. **Flexibility and Adaptability:** 
   The iterative nature of Scrum allows teams to respond quickly to changing requirements, reducing the risk of project failure and ensuring alignment with customer needs.

3. **Focus on Delivering Value:** 
   By emphasizing the delivery of potentially shippable increments, Scrum enables early valid